# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set_style('darkgrid')
sns.set_palette('muted')

DATA_PATH = '../../'

import plotly.graph_objects as go

In [2]:
RANDOM_STATE = 4321

df = pd.read_pickle(DATA_PATH + 'data/raw/historic_btc_15min_interval/all_data.pkl').copy()

# Data Preparation

In [3]:
df['SMA50'] = df['close'].rolling(window=50).mean()
df['SMA200'] = df['close'].rolling(window=200).mean()

df = df.dropna()

df.sort_values(by='time', inplace=True)
df.reset_index(drop=True, inplace=True)

df['signal'] = 0
df['signal'] = np.where(df['SMA50'] > df['SMA200'], 1, 0)
df['position'] = df['signal'].diff().fillna(0)

# Simulation

In [4]:
idxs_tuples = []
first_tuple = True

for idx, pos in zip(df.index, df['position']):
    if pos == 1 and first_tuple:
        first_tuple = False
        idxs_tuples.append((idx, pos))
    elif pos == -1 and not first_tuple:
        first_tuple = True
        idxs_tuples.append((idx, pos))
        
if len(idxs_tuples) % 2 != 0:
    idxs_tuples = idxs_tuples[:-1]

In [5]:
initial_invesment = 1_000
profits_list = []

for i in range(0, len(idxs_tuples), 2):
    id1, id2 = idxs_tuples[i][0], idxs_tuples[i+1][0]
    
    enter_price = df.iloc[id1]['close'] * (1-0.004)
    exit_price = df.iloc[id2]['close'] * (1-0.004)
    
    profit_percentage = (exit_price - enter_price) / enter_price
    
    profits_list.append(profit_percentage * initial_invesment)

## Visualization

In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['time'], y=df['close'], mode='lines', name='Close Price'))

fig.add_trace(go.Scatter(x=df['time'], y=df['SMA50'], mode='lines', name='SMA50'))
fig.add_trace(go.Scatter(x=df['time'], y=df['SMA200'], mode='lines', name='SMA200'))

fig.add_trace(go.Scatter(
    x=df[df['position'] == 1]['time'], 
    y=df['SMA50'][df['position'] == 1], 
    mode='markers', name='Buy Signal', 
    marker=dict(color='green', size=20, symbol='triangle-up')
))

fig.add_trace(go.Scatter(
    x=df[df['position'] == -1]['time'], 
    y=df['SMA50'][df['position'] == -1], 
    mode='markers', name='Sell Signal', 
    marker=dict(color='red', size=20, symbol='triangle-down')
))



fig.update_layout(
    width=1200,
    height=600,
    xaxis_title='Date',
    yaxis_title='Price',
    plot_bgcolor='rgb(40, 40, 40, 100)',
    paper_bgcolor='rgb(40, 40, 40, 100)',
    font=dict(color='rgb(200, 200, 200, 100)'),
    xaxis=dict(
        showline=True,
        linecolor='rgb(200, 200, 200, 100)',
        linewidth=2,
        gridcolor='rgb(200, 200, 200, 100)',
        zerolinecolor='rgb(200, 200, 200, 100)'
    ),
    yaxis=dict(
        showline=True,
        linecolor='rgb(200, 200, 200, 100)',
        linewidth=2,
        gridcolor='rgb(200, 200, 200, 100)',
        zerolinecolor='rgb(200, 200, 200, 100)'
    )
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [7]:
np.sum(profits_list)

np.float64(45.87166021427008)